# Exploration 1

What if we start small (let $\ell = m = n = 3$ so that we are tracking three variables with three neurons) and attempt to train the neural net to learn the Kalman filter? What does the loss look like over time for various initial values of $M$?

In [ ]:
# Import everything needed
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy
import sklearn
import tqdm

from hm_process import HMProcess, plot_hm_process
from kalman import SteadyStateKalmanFilter
from rnn import NeuralNet
from utils import calc_loss, mk_rand_matrix, mk_rand_matrix_envals

rng = np.random.default_rng()

In [ ]:
# Define some arbitrary parameters
def create_process_3d(rng, A=None):
    if A is None:
        A = np.diag([0.98, 0.96, 0.94])
    x0 = np.array([100, 100, 100])
    sigma_process = 10
    Sigma_process = (sigma_process**2)*np.eye(3)
    O = np.eye(3)
    sigma_obs = 20
    Sigma_obs = (sigma_obs**2)*np.eye(3)
    num_steps = 100
    return HMProcess(rng, A, x0, Sigma_process, O, Sigma_obs, num_steps)

proc = create_process_3d(rng)

In [ ]:
ts, xs, ys = proc.simulate()
plot_hm_process('A hidden Markov process governed by linear dynamics', ts, xs, ys)

In [ ]:
# Now, what if we try to predict what the latent states were based
# on the observations by using a steady-state Kalman filter, assuming
# we know the true values of A, Sigma_process, O, and Sigma_obs?
ts, xs, ys = proc.simulate()
kf = SteadyStateKalmanFilter(proc)
xhats = kf.infer(ys)

# Now, let's plot it!
plot_hm_process('A hidden Markov process governed by linear dynamics', ts, xs, ys)
plot_hm_process('Steady-state Kalman filter predictions', ts, xs, ys, xhats)

In [ ]:
# Let's check how well the Kalman filter converges to the steady-state Kalman filter.
Sigma_infty_dist, K_infty_dist, M_infty_dist, _, _, _ = kf.check_convergence()
plt.plot(np.arange(proc.num_steps), K_infty_dist/np.linalg.norm(kf.K_infty, ord='fro'), label='Kalman gain')
plt.plot(np.arange(proc.num_steps), M_infty_dist/np.linalg.norm(kf.M_infty, ord='fro'), label='Transition')
plt.plot(np.arange(proc.num_steps), Sigma_infty_dist/np.linalg.norm(kf.Sigma_infty, ord='fro'), label='Prediction covariance')
plt.title('Convergence of Kalman filter parameters')
plt.xlabel('Time')
plt.ylabel('Frobenius distance divided by norm of steady state')
plt.xlim(0,10)
plt.legend()
plt.show()


In [ ]:
# Now, let's see if we can train a linear dynamical system to learn the Kalman filter.
# First, let's make the matrix A of the hidden Markov process random.
A = mk_rand_matrix_envals(rng, np.array([0.98, 0.96, 0.94]))
proc = create_process_3d(rng, A)
kf = SteadyStateKalmanFilter(proc)

In [ ]:
# How well does the network do if I make the connectivity matrix
# closer or farther from the Kalman filter theory steady-state value?
def test_M_landscape():
    distances = []
    losses = []
    for _ in range(3):
        M = mk_rand_matrix(rng, 3)
        for i in tqdm.tqdm(range(10)):
            c = i/10
            M2 = (1-c)*M + c*kf.M_infty
            nn = NeuralNet(kf.K_infty, M2, np.eye(3), proc.x0)
            Ls = nn.batch_test(100, proc)
            distances.append(np.linalg.norm(M2 - kf.M_infty, ord='fro'))
            losses.append(np.mean(Ls))
    return np.array(distances), np.array(losses)

#distances, losses = test_M_landscape()

In [ ]:
# Let's try plotting our results
#plt.scatter(distances[:10], losses[:10])
#plt.scatter(distances[10:20], losses[10:20])
#plt.scatter(distances[20:], losses[20:])
#plt.xlabel('Frobenius distance between M and the optimal value')
#plt.ylabel('Average loss (100 simulated trials)')
#plt.title('Loss vs distance from Kalman filter theory')
#plt.show()

In [ ]:
# What happens when we train the neural net on this task?
def compare_nn_kalman():
    nn = NeuralNet(0.1*mk_rand_matrix(rng, 3), 0.1*mk_rand_matrix(rng, 3), np.eye(3), proc.x0)
    ts, xs, ys = proc.simulate()
    
    _, xhats = nn.forward(ys)
    plot_hm_process('Before training', ts, xs, ys, xhats)
    print('Loss', calc_loss(xhats, xs))
    print('M =', nn.M)
    print('K =', nn.K)
    
    losses = nn.train(np.array([1e-6]*10), 40, proc)
    plt.plot(losses)
    plt.title('Loss over time')
    plt.xlabel('Batch')
    plt.ylabel('Mean loss')
    plt.show()
    
    _, xhats = nn.forward(ys)
    plot_hm_process('After training', ts, xs, ys, xhats)
    print('Loss', calc_loss(xhats, xs))
    print('M =', nn.M)
    print('K =', nn.K)
    
    # How does it compare to a Kalman filter?
    xhats = kf.infer(ys)
    plot_hm_process('Kalman filter', ts, xs, ys, xhats)
    print('Loss', calc_loss(xhats, xs))
    print('M =', kf.M_infty)
    print('K =', kf.K_infty)

#compare_nn_kalman()

# Exploration 2

What's the right measure of distance between $(M,K)$ and $A$ that corresponds to the average loss at the task? We'll assume $O$ and $W$ are the identity. Can we plot a loss landscape for the task?

In [ ]:
# Let's start with 1 dimension for easy visualization
def create_1d_process(rng, A):
    x0 = np.array([100])
    sigma_process = 10
    Sigma_process = (sigma_process**2)*np.eye(1)
    O = np.array([[1]])
    sigma_obs = 20
    Sigma_obs = (sigma_obs**2)*np.eye(1)
    num_steps = 50
    return HMProcess(rng, A, x0, Sigma_process, O, Sigma_obs, num_steps)

batch_size = 5
num_a_ticks = 5
num_ticks = 20
a_vals = np.array([-0.95, -0.8, 0, 0.8, 0.95])
num_a_ticks = a_vals.shape[0]
mk_range = 0.99
num_trials_per = 40
W = np.array([[1]])
start_from = 5

In [ ]:
def find_nn_values_A_M_K_1d(num_nns=2):
    mks = [] # mks[i][j] contains the values of M and K for the jth trial with the ith value of A

    for i, a_ in enumerate(a_vals):
        A = np.array([[a_]])
        mks_i = []
        for j in tqdm.tqdm(range(num_nns)):
            proc = create_1d_process(rng, A)
            nn = NeuralNet(0.1*mk_rand_matrix(rng, 1), 0.1*mk_rand_matrix(rng, 1), np.eye(1), proc.x0)
            _, _, Ms, Ks = nn.train_until_converge(5e-7, 200, num_trials_per, proc, start_from,
                                                   print_loss=False, progress_bar=False)
            mks_i.append((Ms.flatten(), Ks.flatten()))
        mks.append(mks_i)

    return mks

#mks = find_nn_values_A_M_K_1d()

In [ ]:
def test_A_M_K_loss_landscape_1d():
    losses = np.zeros((num_a_ticks,num_ticks,num_ticks))
    W = np.array([[1]])
    
    for i1, a_ in tqdm.tqdm(enumerate(a_vals), total=num_a_ticks):
        for i2, m_ in enumerate(np.linspace(-1*mk_range,mk_range,num_ticks)):
            for i3, k_ in enumerate(np.linspace(-1*mk_range,mk_range,num_ticks)):
                A = np.array([[a_]])
                M = np.array([[m_]])
                K = np.array([[k_]])
                proc = create_1d_process(rng, A)
                nn = NeuralNet(M, K, W, proc.x0)
                Ls = nn.batch_test(batch_size, proc, start_from)
                losses[i1,i2,i3] = np.mean(Ls)
    
    return losses

#losses = test_A_M_K_loss_landscape_1d()

In [ ]:
#norm = matplotlib.colors.Normalize(3, 4.8)
#for i1, a_ in enumerate(a_vals):
#
#    # Loss landscape
#    c = plt.imshow(np.log10(losses[i1].T), origin='lower', extent=(-1*mk_range,mk_range,-1*mk_range,mk_range), norm=norm)
#    plt.xlabel('M')
#    plt.ylabel('K')
#
#    # Where does the Kalman filter fall?
#    A = np.array([[a_]])
#    proc = create_1d_process(rng, A)
#    kf = SteadyStateKalmanFilter(proc)
#    plt.plot(kf.M_infty, kf.K_infty, color='black', marker='o')
#    plt.annotate('Kalman filter', (kf.M_infty, kf.K_infty),
#                 xytext=(kf.M_infty-0.2, kf.K_infty+0.05), color='black')
#
#    # What about the RNNs?
#    colors = ['red', 'tomato']
#    for j in range(2):
#        plt.plot(mks[i1][j][0], mks[i1][j][1], color=colors[j], linestyle='-', marker='o', markersize=3)
#    
#    plt.title(f'(M, K) Log-Loss Landscape For A={a_:.2f}')
#    plt.xlim(-1*mk_range, mk_range)
#    plt.ylim(-1*mk_range, mk_range)
#    plt.colorbar(c)
#    plt.show()

In [ ]:
#def test_mk(proc, m_, k_):
#    A, M, K = proc.A, np.array([[m_]]), np.array([[k_]])
#    nn = NeuralNet(M, K, W, proc.x0)
#    proc = create_1d_process(rng, A)
#    losses = nn.batch_test(100, proc)
#    print(np.mean(losses))

#A = np.array([[a_vals[-1]]])
#proc = create_1d_process(rng, A)
#kf = SteadyStateKalmanFilter(proc)

#m_ = mks[-1][0][0][-1]
#k_ = mks[-1][0][1][-1]
#test_mk(proc, m_, k_)
#test_mk(proc, kf.M_infty, kf.K_infty)

#ts, xs, ys = proc.simulate()

#nn = NeuralNet(np.array([[m_]]), np.array([[k_]]), np.eye(1), proc.x0)
#_, xhats = nn.forward(ys)
#plot_hm_process('After training', ts, xs, ys, xhats)
#print('Loss', calc_loss(xhats, xs, start_from))
#print('M =', nn.M)
#print('K =', nn.K)

#xhats = kf.infer(ys)
#plot_hm_process('Kalman filter', ts, xs, ys, xhats)
#print('Loss', calc_loss(xhats, xs, start_from))
#print('M =', kf.M_infty)
#print('K =', kf.K_infty)

In [ ]:
# What about two dimensions?
def create_2d_process(rng, A):
    x0 = np.array([100, 100])
    sigma_process = 5
    Sigma_process = (sigma_process**2)*np.eye(2)
    O = np.eye(2)
    sigma_obs = 20
    Sigma_obs = (sigma_obs**2)*np.eye(2)
    num_steps = 50
    return HMProcess(rng, A, x0, Sigma_process, O, Sigma_obs, num_steps)

batch_size = 5
num_a_ticks = 5
num_ticks = 20
A = mk_rand_matrix_envals(rng, np.array([0.9, 0.97]))
proc = create_2d_process(rng, A)
kf = SteadyStateKalmanFilter(proc)
mk_range = 0.99
num_trials_per = 40
W = np.eye(2)
start_from = 5

In [ ]:
def test_K_loss_landscape_2d():
    M = kf.M_infty
    
    losses = np.zeros((num_ticks,num_ticks))    
    for i_k1, k_ev1 in tqdm.tqdm(enumerate(np.linspace(-1*mk_range,mk_range,num_ticks)), total=num_ticks):
        for i_k2, k_ev2 in enumerate(np.linspace(-1*mk_range,mk_range,num_ticks)):
            K = mk_rand_matrix_envals(rng, np.array([k_ev1, k_ev2]))
            nn = NeuralNet(M, K, W, proc.x0)
            Ls = nn.batch_test(batch_size, proc, start_from)
            losses[i_k1,i_k2] = np.mean(Ls)
    
    return losses

#losses_K = test_K_loss_landscape_2d()

In [ ]:
def test_M_loss_landscape_2d():
    K = kf.K_infty
    
    losses = np.zeros((num_ticks,num_ticks))    
    for i_m1, m_ev1 in tqdm.tqdm(enumerate(np.linspace(-1*mk_range,mk_range,num_ticks)), total=num_ticks):
        for i_m2, m_ev2 in enumerate(np.linspace(-1*mk_range,mk_range,num_ticks)):
            M = mk_rand_matrix_envals(rng, 2, np.array([m_ev1, m_ev2]))
            nn = NeuralNet(M, K, W, proc.x0)
            Ls = nn.batch_test(batch_size, proc, start_from)
            losses[i_m1,i_m2] = np.mean(Ls)
    
    return losses

#losses_M = test_M_loss_landscape_2d()

In [ ]:
def test_M_loss_landscape_2d_diagentry():
    K = kf.K_infty
    M = kf.K_infty
    
    losses = np.zeros((num_ticks,num_ticks))    
    for i_m1, m1 in tqdm.tqdm(enumerate(np.linspace(-1*mk_range,mk_range,num_ticks)), total=num_ticks):
        for i_m2, m2 in enumerate(np.linspace(-1*mk_range,mk_range,num_ticks)):
            M[0,0] = m1
            M[1,1] = m2
            nn = NeuralNet(M, K, W, proc.x0)
            Ls = nn.batch_test(batch_size, proc, start_from)
            losses[i_m1,i_m2] = np.mean(Ls)
    
    return losses

#losses_M_diag = test_M_loss_landscape_2d_diagentry()

In [ ]:
# Loss landscape
#norm = matplotlib.colors.Normalize(4, 6)
#c = plt.imshow(np.log10(losses_M_diag.T), origin='lower', extent=(-1*mk_range,mk_range,-1*mk_range,mk_range), norm=norm)
#plt.xlabel('M diag 1')
#plt.ylabel('M diag 2')

# Where does the Kalman filter fall?
#envals = (kf.M_infty[0,0], kf.M_infty[1,1])
#plt.plot(envals[0], envals[1], color='black', marker='o')
#plt.annotate('Kalman filter', (envals[0], envals[1]),
#             xytext=(envals[0]-0.2, envals[1]+0.05), color='black')

# What about the RNNs?
#plt.scatter(mks[i1,:,0], mks[i1,:,1], color='red', marker='o')

#plt.title(f'M Log-Loss Landscape')
#plt.xlim(-1*mk_range, mk_range)
#plt.ylim(-1*mk_range, mk_range)
#plt.colorbar(c)
#plt.show()

In [ ]:
#kf = SteadyStateKalmanFilter(proc)
#ts, xs, ys = proc.simulate()

#nn = NeuralNet(np.diag((0,0.5)), kf.K_infty, np.eye(2), proc.x0)
#_, xhats = nn.forward(ys)
#plot_hm_process('After training', ts, xs, ys, xhats)
#print('Loss', calc_loss(xhats, xs, start_from))
#print('M =', nn.M)
#print('K =', nn.K)

#xhats = kf.infer(ys)
#plot_hm_process('Kalman filter', ts, xs, ys, xhats)
#print('Loss', calc_loss(xhats, xs, start_from))
#print('M =', kf.M_infty)
#print('K =', kf.K_infty)

#print(kf.Sigma_infty)

# Exploration 3
The goal of this exploration is to work out some possible bugs with the implementation of the Kalman filter (why does the covariance matrix not converge properly?) and find an appropriate method to systematically determine values of the buy-in period (at what time step do we start the loss calculation?) and input gain.

First bug solved: the issue was that I was plotting the distance between the posterior estimate covariance and the steady-state estimate covariance when I should have been plotting the distance between the prior estimate covariance and the steady-state estimate covariance instead. Now the graph looks as expected.

Let's experiment next with simulating 1D processes and checking how long the buy-in period is.

In [ ]:
def create_process_1d_more_parameters(rng, a, sigma_process, sigma_obs):
    A = np.array([[a]])
    x0 = np.array([100])
    Sigma_process = (sigma_process**2)*np.eye(1)
    O = np.eye(1)
    Sigma_obs = (sigma_obs**2)*np.eye(1)
    num_steps = 50
    return HMProcess(rng, A, x0, Sigma_process, O, Sigma_obs, num_steps)

def time_of_convergence(proc, eps=0.05, i=0):
    kf = SteadyStateKalmanFilter(proc)
    Sigma_infty_dist, K_infty_dist, M_infty_dist, _, _, _ = kf.check_convergence()
    for i in range(proc.num_steps-1,-1,-1):
        if Sigma_infty_dist[i] > eps:
            return i+1
        if K_infty_dist[i] > eps:
            return i+1
        if M_infty_dist[i] > eps:
            return i+1
    return 0

def plot_1d_kalman_convergence_dynamics():
    num_points = 100
    a_vals = np.linspace(0.01,0.99,num_points)
    t_vals = np.zeros(num_points)
    for i in range(num_points):
        proc = create_process_1d_more_parameters(rng, a_vals[i], 10, 25)
        t_vals[i] = time_of_convergence(proc)
    plt.plot(a_vals, t_vals, label='Actual trend')
    plt.plot(a_vals, -1/np.log(a_vals), label='Proportional to time constant')
    plt.plot(a_vals, 11*a_vals+2, label='Roughly linear')
    plt.title('Length of buy-in period vs largest eigenvalue (1D)')
    plt.xlabel('Largest eigenvalue of A')
    plt.ylabel('Time until <5% error')
    plt.ylim(0,12)
    plt.legend()
    plt.show()

plot_1d_kalman_convergence_dynamics()

It looks like the fit is not proportional to the time constant of the process, as I would have expected, but instead is roughly linear! Choosing a buy-in time period of eleven times the value of A plus two should be sufficient for letting the Kalman filter converge. What if we try changing the noise parameters?

In [ ]:
def plot_1d_kalman_convergence_noise():
    num_points = 30
    max_proc_noise = 50
    max_obs_noise = 50
    sigma_proc_vals = np.linspace(1,max_proc_noise,num_points)
    sigma_obs_vals = np.linspace(1,max_obs_noise,num_points)
    t_vals = np.zeros((num_points,num_points))
    for i in range(num_points):
        for j in range(num_points):
            proc = create_process_1d_more_parameters(rng, 0.99, sigma_proc_vals[i], sigma_obs_vals[j])
            t_vals[i,j] = time_of_convergence(proc)
    norm = matplotlib.colors.Normalize(0, 11)
    plt.imshow(t_vals, extent=(1,max_obs_noise,1,max_proc_noise), origin='lower', norm=norm)
    x_vals = np.linspace(1, max_obs_noise, 200)
    plt.plot(x_vals, x_vals/2, color='red')
    plt.title('Length of buy-in period vs noise (1D)')
    plt.xlabel('Observation noise standard deviation')
    plt.ylabel('Process noise standard deviation')
    plt.xlim(1, max_obs_noise)
    plt.ylim(1, max_proc_noise)
    plt.colorbar()
    plt.show()

plot_1d_kalman_convergence_noise()

Process and observation noise seem to make the convergence take much longer when the observation noise is high and the process noise is very low. A=0.99 originally took 11 steps, and most of this graph is in the non-yellow region where the number of steps needed is below 11 steps, as long as the observation noise is less than twice the process noise (which is the red line).

Now, let's try it with multi-dimensional processes.

In [ ]:
def create_process_3d_based_on_envals(rng, enval1, enval2, enval3):
    A = mk_rand_matrix_envals(rng, np.array([enval1, enval2, enval3]))
    x0 = np.array([100, 100, 100])
    sigma_process = 10
    Sigma_process = (sigma_process**2)*np.eye(3)
    O = np.eye(3)
    sigma_obs = 25
    Sigma_obs = (sigma_obs**2)*np.eye(3)
    num_steps = 50
    return HMProcess(rng, A, x0, Sigma_process, O, Sigma_obs, num_steps)

def plot_3d_kalman_convergence_dynamics():
    max_envals = []
    t_vals = []
    num_points = 10
    enval_space = np.linspace(0.01,0.99,num_points)
    for enval1 in enval_space:
        for enval2 in enval_space:
            for enval3 in enval_space:
                try:
                    proc = create_process_3d_based_on_envals(rng, enval1, enval2, enval3)
                    t_vals.append(time_of_convergence(proc))
                    max_envals.append(max(enval1, enval2, enval3))
                except sklearn.exceptions.ConvergenceWarning:
                    pass
    
    plt.scatter(max_envals, t_vals)
    plt.title('Length of buy-in period vs largest eigenvalue (3D)')
    plt.xlabel('Largest eigenvalue of A')
    plt.ylabel('Time until <5% error')
    plt.show()

plot_3d_kalman_convergence_dynamics()

The graph has roughly the same shape as in the 1D case, which is reassuring, although there are some matrices which are outliers. Let's check one of them by hand. All in all, I conclude that if the process noise is more than half the observation noise, then a buy-in period of a little bit more than 10 times the largest eigenvalue of the dynamics matrix should suffice.

Next, how do we deal with the input gain?

In [ ]:
def find_appropriate_gain(proc, M):
    # return best K?
    pass

def plot_gain_function():
    norm = matplotlib.colors.Normalize(3, 4.8)
    for i1, a_ in enumerate(a_vals):
    
        # Loss landscape
        c = plt.imshow(np.log10(losses[i1].T), origin='lower', extent=(-1*mk_range,mk_range,-1*mk_range,mk_range), norm=norm)
        plt.xlabel('M')
        plt.ylabel('K')
    
        # Best values for K, given M?
        ms = np.linspace(-1*mk_range, mk_range, 100)
        ks = []
        proc = create_1d_process(rng, np.array([[a_]]))
        for m_ in ms:
            K = find_appropriate_gain(proc, np.array([[m_]]))
            ks.append(K[0,0])
        ks = np.array(ks)
        plt.plot(ms, ks, color='red')
        
        plt.title(f'(M, K) Log-Loss Landscape For A={a_:.2f}')
        plt.xlim(-1*mk_range, mk_range)
        plt.ylim(-1*mk_range, mk_range)
        plt.colorbar(c)
        plt.show()

#plot_gain_function()

# Exploration 4
Now, what if we add some structure onto our connectivity matrix?

In [ ]:
# Let's see if the loss is impacted by random masks

def mk_rand_mask(num_neurons=3):
    return np.random.randint(0,2,size=(num_neurons,num_neurons))

def test_masks():
    for _ in range(2):
        mask = mk_rand_mask()
        print(mask)
        nn = NeuralNet(0.1*mk_rand_matrix(rng, 3), 0.1*mk_rand_matrix(rng, 3), np.eye(3), proc.x0, mask=mask)
        num_batches, losses = nn.train_until_converge(1e-6, 1000, 20, proc)
        plt.plot(losses)
        plt.xlabel('Batch')
        plt.ylabel('Mean loss')
        plt.show()

#test_masks()